## **1. VWAP Optimal Execution Model Code**

In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the CSV file
file_path = 'xnas-itch-20230703.tbbo.csv'
data = pd.read_csv(file_path)

# Convert ts_event to datetime and extract minute-level data
data['ts_event'] = pd.to_datetime(data['ts_event'], unit='ns')
data.set_index('ts_event', inplace=True)

# Resample to minute-level data
minute_data = data.resample('T').agg({
    'price': 'mean',
    'size': 'sum',
    'bid_px_00': 'last',
    'ask_px_00': 'last',
    'bid_sz_00': 'last',
    'ask_sz_00': 'last'
}).dropna()

# Calculate the VWAP for each minute
minute_data['vwap'] = (minute_data['price'] * minute_data['size']).cumsum() / minute_data['size'].cumsum()

# Calculate the total daily VWAP
total_volume = minute_data['size'].sum()
total_vwap = (minute_data['price'] * minute_data['size']).sum() / total_volume

# Calculate the fractional bid-ask spread (s)
minute_data['spread'] = minute_data['ask_px_00'] - minute_data['bid_px_00']
minute_data['mid_price'] = (minute_data['ask_px_00'] + minute_data['bid_px_00']) / 2
minute_data['fractional_spread'] = minute_data['spread'] / minute_data['mid_price']

# Calculate the average fractional bid-ask spread
s = minute_data['fractional_spread'].mean()

# Estimate the proportionality factor (alpha)
# Simplified approach: Regression of price change on volume
minute_data['price_change'] = minute_data['price'].diff()
minute_data['volume_ratio'] = minute_data['size'] / minute_data['size'].sum()

# Drop the first row with NaN value from diff() operation
minute_data = minute_data.dropna()

# Prepare data for regression
X = minute_data['volume_ratio']
y = minute_data['price_change']
X = sm.add_constant(X)  # Adds a constant term to the predictor

# Fit the regression model
model = sm.OLS(y, X).fit()
alpha = model.params[1]  # The slope coefficient

# Update the effective price calculation with new s and alpha
minute_data['effective_price'] = minute_data['price'] * (1 - s/2 + alpha * s/2 * (minute_data['size'] / minute_data['size'].sum()))
minute_data['slippage'] = (minute_data['effective_price'] - total_vwap) / total_vwap

# Total slippage for the day with new parameters
total_slippage = minute_data['slippage'].sum()

# Print the first few rows and the total slippage
minute_data[['price', 'size', 'vwap', 'effective_price', 'slippage']].head(), total_slippage


(                            price  size          vwap  effective_price  \
 ts_event                                                                 
 2023-07-03 08:01:00  1.942083e+11  1846  1.940841e+11    -2.865572e+12   
 2023-07-03 08:02:00  1.941918e+11   305  1.940884e+11    -3.113241e+11   
 2023-07-03 08:03:00  1.941738e+11   326  1.940920e+11    -3.460966e+11   
 2023-07-03 08:04:00  1.941947e+11   795  1.941014e+11    -1.123451e+12   
 2023-07-03 08:05:00  1.941519e+11   566  1.941045e+11    -7.437441e+11   
 
                       slippage  
 ts_event                        
 2023-07-03 08:01:00 -15.873604  
 2023-07-03 08:02:00  -2.615912  
 2023-07-03 08:03:00  -2.796397  
 2023-07-03 08:04:00  -6.831216  
 2023-07-03 08:05:00  -4.860366  ,
 -70504.22030060011)

## **2. Kyle's model (extended)**

In [6]:
# Convert timestamps to a more readable format
data['ts_recv'] = pd.to_datetime(data['ts_recv'], unit='ns')
data['ts_event'] = pd.to_datetime(data['ts_event'], unit='ns')

# Calculate signed order flow
data['signed_order_flow'] = np.where(data['side'] == 'B', data['size'], -data['size'])

# Calculate price changes
data['price_change'] = data['price'].diff()

# Calculate time differences in seconds between consecutive events
data['time_diff'] = data['ts_event'].diff().dt.total_seconds()

# Drop NaN values resulting from the diff operations
data = data.dropna(subset=['price_change', 'time_diff'])

# Regress price changes on signed order flow to obtain Kyle’s lambda
X = data['signed_order_flow']
y = data['price_change']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
kyle_lambda = model.params[1]

# Calculate volatility of the order flow
data.loc[:, 'volatility'] = data['signed_order_flow'].rolling(window=100).std()

# Drop NaN values resulting from rolling operation
data = data.dropna(subset=['volatility'])

# Calculate the execution cost with time differences
data.loc[:, 'execution_cost'] = kyle_lambda * (data['volatility'] ** 2) * data['time_diff']
total_execution_cost = data['execution_cost'].sum()

total_execution_cost


-2614571472320.662